In [ ]:
%load_ext autoreload
%autoreload 2
%gui qt

In [ ]:
import os
import numpy as np
from pya import Asig, Esig

# TEST_FILE = os.path.join(os.path.abspath('..'), "data\\stereoTest.wav")
# signal = Asig(TEST_FILE)
signal = Asig(np.sin(2* np.pi * 440 * np.linspace(0, 1, 44100)))
edit_signal = Esig(signal.mono())

In [ ]:

from pyavis.graphics import createAxis, createMultiTrack

mt = createMultiTrack()
track = mt.add_track("test_1", 44100)

mt.show()

In [ ]:


def midi_conv(value) -> str:
    midi_value = int(value)
    notes = ['C', 'C#', 'D', 'D#', 'E', 'F', 'F#', 'G', 'G#', 'A', 'A#', 'B',]
    octave = int(midi_value / 12 - 1)
    return f'{notes[midi_value % 12]}{octave}'

ax_left = createAxis("left", lambda value: f'{round(value,1)} | {midi_conv(value)}')
ax_left.set_spacing((5,1))
ax_bottom = createAxis("bottom", lambda value: f'{round(value / track.sampling_rate, 2)}') 

In [ ]:
track.set_axis(ax_left)
track.set_axis(ax_bottom)

In [ ]:
sig = track.add_signal((0,69), edit_signal.asig.sig)
# Better naming scheme, e.g. set_y_range
sig.set_size(1)

for event in edit_signal.cache.events:
    rect = track.add_rectangle((int(event.start), 68.5), (int((event.end - event.start)), 1))
    rect.set_style(style={"fill": (0,255,0, 25), "border": (255,0,0, 255)})
    rect.draggable = True
    rect.draggingFinish.connect(lambda args: move_sig(sig, args))

In [ ]:
track.set_view_limits(xMin = 0, xMax=len(sig.data), yMin = 0, yMax=130)

In [ ]:
def move_sig(signal, args):
    new_x_value = args[1][0]
    new_y_value = args[1][1]
    sig.set_position(new_x_value, new_y_value + 5)

    midi = round(new_y_value)
    print(midi)

    # if len(edit_signal.edits) > 0:
    #     edit_signal.undo_last()
    # edit_signal.correct_pitch(0, edit_signal.asig.dur, np.array([(0.9, midi)]))
    
    

In [ ]:
track.allow_dragging(False, True)
track.set_dragging_limits((0, len(sig.data)), ((1, 128)))